Universidad del Valle de Guatemala

Minería de Datos

Proyecto 2 Entrega 5

Wilson Calderón, 22018

Abby Donis, 22440


## 1. Creación de Variables Dicotómicas 

Se ha creado una variable categórica partiendo de SalePrice utilizando cuantiles del 33 y 66% respectivamente. Esto permite clasificar viviendas en tres grupos. Económico, Medio y Caro. De ahí se generaron variables binarias llamadas EsCara, EsMedia y EsEconomica que toman valores 1 o 0 según corresponda. Si una casa se encuentra en el tercil superior, la variable EsCara es 1. De lo contrario es 0.


In [ ]:
def main():
    # ---------------------
    # Inciso 1: Crear Variables Dicotómicas a partir de SalePrice
    # ---------------------
    try:
        df = pd.read_csv("train.csv")
    except Exception as e:
        print("Error al cargar 'train.csv':", e)
        return

    print("Columnas del dataset:", df.columns.tolist())

    # Utilizamos los cuantiles 33% y 66% para definir los umbrales
    quantiles = df['SalePrice'].quantile([0.33, 0.66])
    q_low = quantiles.iloc[0]
    q_high = quantiles.iloc[1]

    def categorizar_precio(x):
        if x >= q_high:
            return 'cara'
        elif x >= q_low:
            return 'media'
        else:
            return 'económica'

    df['Categoria'] = df['SalePrice'].apply(categorizar_precio)
    df['EsCara'] = (df['Categoria'] == 'cara').astype(int)
    df['EsMedia'] = (df['Categoria'] == 'media').astype(int)
    df['EsEconomica'] = (df['Categoria'] == 'económica').astype(int)

    print("\nPrimeras filas con SalePrice, Categoria y variables dicotómicas:")
    print(df[['SalePrice', 'Categoria', 'EsCara', 'EsMedia', 'EsEconomica']].head())

# 2. Mismos conjuntos de entrenamiento y prueba
Mismos conjuntos de entrenamiento y prueba
Para garantizar la reproducibilidad del experimento, el dataset train.csv se dividió en conjuntos de entrenamiento del 70% y prueba del 30%. Usando la función train_test_split con random_state = 42. Así se mantiene siempre las mismas particiones en cada ejecución.

In [ ]:
    # ---------------------
    # Inciso 3: Modelo de Regresión Logística con Validación Cruzada
    # ---------------------
modelo = LogisticRegression(max_iter=1000, solver='liblinear')
cv_scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
print("\nExactitud promedio en validación cruzada:", np.mean(cv_scores))
    
    # Entrena el modelo con el conjunto de entrenamiento completo
modelo.fit(X_train, y_train)

# 3. Modelo de regresión logística
Se utiliza un modelo de regresión logística para predecir si una vivienda es cara (variable EsCara). Este modelo ha sido entrenado con el conjunto entrenamiento y se aplicó validación cruzada (5-fold). Lo que produjo una exactitud promedio del 88.85%. Esto indica que el modelo se comporta de forma consistente.

In [ ]:
# ---------------------
# Inciso 3: Modelo de Regresión Logística con Validación Cruzada
# ---------------------
modelo = LogisticRegression(max_iter=1000, solver='liblinear')
cv_scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
print("\nExactitud promedio en validación cruzada:", np.mean(cv_scores))
    
# Entrena el modelo con el conjunto de entrenamiento completo
modelo.fit(X_train, y_train)

# 4. Análisis del modelo
Para evaluar el modelo se realizó un análisis de multicolinealidad mediante la matriz de correlación (ver imagen de abajo). Esta gráfica no muestra correlaciones excesivamente fuertes, por lo que es mejor para el modelo. Se examinaron OverallQual y GarageCars. Tienen coeficientes positivos y significativos, mientras que YearBuilt presenta un coeficiente negativo. La dirección y magnitud de los coeficientes, sin valores p formales, indican que algunas variables aportan de forma notable al modelo.



In [ ]:
# ---------------------
# Inciso 4: Análisis del Modelo 
# ---------------------
# 4.1: Análisis de multicolinealidad mediante la matriz de correlación
corr_matrix = X_train.corr()
plt.figure(figsize=(8, 6))
plt.imshow(corr_matrix, cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns)
plt.title("Matriz de correlación de variables predictoras")
plt.tight_layout()
plt.show()

# 5. Evaluación en el Conjunto de Prueba
Al aplicar el modelo sobre el conjunto de prueba se obtuvo una matriz de confusión con 282 verdaderos negativos, 122 verdaderos positivos, 14 falsos positivos y 20 falsos negativos. El reporte de clasificación muestra una exactitud global de 92.24%. Esto confirma que el modelo clasifica de forma eficiente. 


In [ ]:
# ---------------------
# Inciso 5: Evaluación del Modelo en el Conjunto de Prueba
# ---------------------
y_pred = modelo.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusión:")
print(cm)
    
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
    
print("Exactitud (accuracy) en el conjunto de prueba:", accuracy_score(y_test, y_pred))

# 6. Sobreajuste y curvas de aprendizaje
Las exactitudes de la validación cruzada y  conjunto de prueba alcanzan un aproximado del 89% y 92.2% respectivamente. En conjunto con que el desempeño para la clase 0 la precisión y el recall llegan al 93% y 95%, en la clase 1 estos valores son de 90% y 86% respectivamente. Además de que el f1-score es del 94% y 88% para su clase respectiva, teniendo un buen balance. 


# 7. Tuneo de modelo 
Al haber aplicado el tuneo para el modelo podemos ver que los mejores parametros fueron L2 para tipo de regularización, con una fuerza de 1 y el algoritmo liblinear. La exactitud logró mejorar al 93.6% indicando que el tuneo fue efectivo al subir del 92.2%

In [ ]:
# ------------------------------
# Inciso 7: Tuneo curva de aprendizaje
# ------------------------------
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=1000))
])
#busca parametro
param_grid = {
'model__penalty': ['l1', 'l2'],
'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
'model__solver': ['liblinear', 'saga']
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Exactitud:", accuracy_score(y_test, y_pred))

# 8. Análisis de eficiencia del algoritmo
Podemos ver que hay un total de 281 verdaderos negativos, 129 verdaderos positivos, 15 13 falsos negativos y 15 falsos positivos. De esto, se marca como error el 5.07% de los falsos positivos y el 9.15% de falsos negativos, al comparar podemos ver que hubo una mejora en los falsos positivos y negativos. 
En cuanto a los tiempos de ejecución el entrenamiento lleva 0.0235 segundos, bastante óptimo. Las funciones más costosas son train_and_predict (0.024s), el procesamiento de pipeline (0.017s) y la validación de los datos (0.013s), la memoria realiza 9214 llamadas a funciones y hay un uso moderado de recursos según el perfilado.
Completando con la información que acabamos de obtener podemos decir que el modelo es bastante efectivo. Esto ya que hay un buen balance entre precisión (92.85%),  rendimiento (tiempos de ejecución bastante bajos) y estabilidad (bajo consumo de recursos).


In [ ]:
    # ------------------------------
    # Inciso 8: Matriz de confusion y rendimiento
    # ------------------------------
y_pred = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusion (Tuneo):")
print(cm)

print("\n**ANÁLISIS DE RENDIMIENTO**")
# Análisis detallado de errores
tn, fp, fn, tp = cm.ravel()
print("\nDetalle de errores:")
print(f"- Falsos Positivos (FP): {fp} casos - Viviendas normales clasificadas como caras")
print(f"- Falsos Negativos (FN): {fn} casos - Viviendas caras clasificadas como normales")
print(f"\nTasa de error FP: {fp/(fp+tn):.2%}")
print(f"Tasa de error FN: {fn/(fn+tp):.2%}")

# Análisis de rendimiento con cProfile
tn, fp, fn, tp = cm.ravel()
print("\n**DETALLE DE ERRORES**")
print(f"Falsos Positivos (FP): {fp} casos - Viviendas normales clasificadas como caras")
print(f"Falsos Negativos (FN): {fn} casos - Viviendas caras clasificadas como normales")
print(f"\nTasa de error FP: {fp/(fp+tn):.2%}")
print(f"Tasa de error FN: {fn/(fn+tp):.2%}")

# Función para análisis de rendimiento
print("\n**ANÁLISIS DE RENDIMIENTO**")
print("=== Resultados del perfilado ===")
def profile(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        pr = cProfile.Profile()
        pr.enable()
        start_time = time.time()
            
        result = func(*args, **kwargs)
            
        end_time = time.time()
        pr.disable()
            
        print(f"\nTiempo de ejecución: {end_time - start_time:.4f} segundos")
            
        s = io.StringIO()
        ps = pstats.Stats(pr, stream=s).sort_stats('cumtime')
        ps.print_stats(10)
        print(s.getvalue())
            
        return result
    return wrapper

    # Función a analizar
    @profile
    def train_and_predict():
        model = Pipeline([
            ('scaler', StandardScaler()),
            ('model', LogisticRegression(penalty='l2', C=1, solver='liblinear', max_iter=1000))
        ])
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return accuracy_score(y_test, y_pred)
    train_and_predict()

# 9. Mejor modelo
Comparando modelos desarrollados, se evaluaron tanto las métricas de desempeño como exactitud, precisión y más con criterios de complejidad AIC y BIC con tiempos de ejecución. Con esto, vemos que el modelo de Regresión Logística optimizado mostró un buen equilibrio entre precisión, eficiencia y complejidad. 
Podemos ver que el modelo tiene un excelente desempeño con métricas AIC y BIC bajas, indicando un buen ajuste estadístico.  


In [ ]:
# ------------------------------
# Inciso 9: Mejor modelo
# ------------------------------
aic_bin, bic_bin = aic_bic(best_model.named_steps['model'], X_test, y_test)
print("\nMetricas estadisticas:")
print(f"AIC: {aic_bin:.2f} | BIC: {bic_bin:.2f}")
print("(Valores mas bajos indican mejor modelo)")
#matriz
y_pred_bin = best_model.predict(X_test)
cm_bin = confusion_matrix(y_test, y_pred_bin)
tn, fp, fn, tp = cm_bin.ravel()
    
print("\nAnálisis de errores:")
print(f"FP (Sobrestimación): {fp} casos ({fp/(fp+tn):.2%})")
print(f"FN (Subestimación): {fn} casos ({fn/(fn+tp):.2%})")

# 10. Regresión logística para variable categórica
Se implementó una regresión logística para predecir el precio de las viviendas entre baratas, medias y caras. Luego de ajustar los hiper-parámetros mediante validación cruzada, se mostró un buen desempeño. Esto se ve en la matriz de confusión y reporte de clasificación. Sin embargo, podemos ver cierta dificultad con la clasificación de casas en la categoría “cara” al tener un recall bastante bajo de 52% y un f1-score del 62%. 

In [ ]:
# ------------------------------
# Inciso 10: Regresion para variable de precios
# ------------------------------
X_multi = df[features]
y_multi = LabelEncoder().fit_transform(df['Categoria'])  # 0:barata, 1:media, 2:cara
    
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multi, y_multi, test_size=0.3, random_state=42)
    
# Pipeline y parametros
 multi_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', OneVsRestClassifier(LogisticRegression(max_iter=1000)))
])
param_grid = {
    'model__estimator__C': [0.1, 1, 10],
    'model__estimator__penalty': ['l2'],
    'model__estimator__solver': ['liblinear', 'saga']
}
print("\nTUNEADO MULTICASE")
grid_multi = GridSearchCV(multi_pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_multi.fit(X_train_m, y_train_m)
    
best_multi = grid_multi.best_estimator_
print("\nMejores parametros:", grid_multi.best_params_)

y_pred_multi = best_multi.predict(X_test_m)
print("\nReporte clasificacion:")
print(classification_report(y_test_m, y_pred_multi, target_names=['Económica', 'Media', 'Cara']))

# 11. Comparación de eficiencia de modelos
Se compararon los modelos anteriores y este. La comparación reveló cuál modelo logra un mejor balance entre eficiencia y precisión de esta clasificación. En base a los resultados, el modelo binario es más rápido y preciso, esto puede explicarse gracias a que este es más simple. Aunque cabe destacar que el Multicase de un poco de información más detallada.

In [ ]:
# ------------------------------
# Inciso 11: Comparación de eficiencia de modelos
# ------------------------------
@profile
def evaluate_model(model, X_train, y_train, X_test):
    start_e = time.time()
    model.fit(X_train, y_train)
    train_time = max(time.time() - start_e, 0.000001)
        
    start_p = time.time()
    y_pred = model.predict(X_test)
    pred_time = max(time.time() - start_p, 0.000001)
        
    return {
        'Tiempo Entrenamiento': train_time,
        'Tiempo Predicción': pred_time,
        'Exactitud': accuracy_score(y_test, y_pred)
    }
    
    #Ev y comparacion
    #binario
    res_bin = evaluate_model(best_model, X_train, y_train, X_test)
    #multicase
    res_multi = evaluate_model(best_multi, X_train_m, y_train_m, X_test_m)
    
    comparison = pd.DataFrame({
        'Binario': res_bin,
        'Multiclase': res_multi
    }).T
    print("\nTabla comparativa:")
    print(comparison)
    
    print("\n*** CONCLUSIONES ***")
    print("Rendimiento computacional:")
    print("  - Tiempo (mas lento):")
    print(f"   Entrenamiento: {res_multi['Tiempo Entrenamiento']/res_bin['Tiempo Entrenamiento']:.1f}x")
    print(f"   Prediccion:    {res_multi['Tiempo Predicción']/res_bin['Tiempo Predicción']:.1f}x")
    
    print("\nPrecisión:")
    print(f"   Binario: {res_bin['Exactitud']:.2%} de exactitud")
    print(f"   Multiclase: {res_multi['Exactitud']:.2%} de exactitud")